In [126]:
import numpy as np
import plotly.graph_objects as go
from scipy.io import wavfile
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

def importWAV(filename):
    samplerate, rawData = wavfile.read(filename)
    
    time = np.linspace(0, rawData.shape[0]/samplerate, rawData.shape[0])   
    
    data = {'left':rawData[:, 0],'right':rawData[:, 1]}
    return time,data

In [127]:
v_100 = importWAV('40mV_100Hz.wav')
v_500 = importWAV('40mV_500Hz.wav')
v_1k = importWAV('40mV_1kHz.wav')
v_2k = importWAV('40mV_2kHz.wav')
v_4k = importWAV('40mV_4kHz.wav')
v_6k = importWAV('40mV_6kHz.wav')
v_8k = importWAV('40mV_8kHz.wav')
v_11k = importWAV('40mV_11kHz.wav')
v_13k = importWAV('40mV_13kHz.wav')
v_15k = importWAV('40mV_15kHz.wav')
v_18k = importWAV('40mV_18kHz.wav')

t_100 = v_100[0]
t_500 = v_500[0]
t_1k = v_1k[0]
t_2k = v_2k[0]
t_4k = v_4k[0]
t_6k = v_6k[0]
t_8k = v_8k[0]
t_11k = v_11k[0]
t_13k = v_13k[0]
t_15k = v_15k[0]
t_18k = v_18k[0]

t_arr = np.array([t_100, t_500, t_1k, t_2k, t_4k, t_6k, t_8k, t_11k, t_13k, t_15k, t_18k])

v_100 = v_100[1]['right']
v_500 = v_500[1]['right']
v_1k = v_1k[1]['right']
v_2k = v_2k[1]['right']
v_4k = v_4k[1]['right']
v_6k = v_6k[1]['right']
v_8k = v_8k[1]['right']
v_11k = v_11k[1]['right']
v_13k = v_13k[1]['right']
v_15k = v_15k[1]['right']
v_18k = v_18k[1]['right']

v_arr = np.array([v_100, v_500, v_1k, v_2k, v_4k, v_6k, v_8k, v_11k, v_13k, v_15k, v_18k])

f_arr = np.array([1E2, 5E2, 1E3, 2E3, 4E3, 6E3, 8E3, 11E3, 13E3, 15E3, 18E3])

/var/folders/rg/hh4cb_xd5_74c2r0hymqq4jm0000gn/T/ipykernel_6065/1836492366.py:8: WavFileWarning:

Chunk (non-data) not understood, skipping it.

/var/folders/rg/hh4cb_xd5_74c2r0hymqq4jm0000gn/T/ipykernel_6065/3374576093.py:25: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/var/folders/rg/hh4cb_xd5_74c2r0hymqq4jm0000gn/T/ipykernel_6065/3374576093.py:39: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [128]:
global f_c

V_amp_arr = np.sqrt(2) * np.array([29.24E-3, 28.25E-3, 28.25E-3, 28.25E-3, 28.25E-3, 29.23E-3,
                                   28.25E-3, 29.3E-3 ,29.19E-3, 29.19E-3, 29.18E-3])
V_amp_std = 0.02E-3

def fit_sin(t, a, b):
    global f_c
    global V_amp
    return a * V_amp * np.sin(f_c * (2*np.pi) * t + b)

C_arr = np.array([])
C_std_arr = np.array([])

for v, f, V_amp in zip(v_arr, f_arr, V_amp_arr):
    n = round(6.5 * f)
    v_c = np.copy(v)
    v_sorted = np.sort(v_c)
    amps = (v_sorted[(-n - 1): -51] - v_sorted[50: n]) / 2
    C = np.mean(amps) / V_amp
    C_std = 1 / V_amp * np.sqrt((np.std(amps)) ** 2 + (np.mean(amps) * V_amp_std / V_amp) ** 2)
    
    C_arr = np.append(C_arr, C)
    C_std_arr = np.append(C_std_arr, C_std)

[9.30619856 9.27417456 9.21252827 9.06028117 8.94574543 9.04000443
 8.70560835 8.30319599 8.11690836 7.54530598 7.19672099] [0.00495552 0.00626411 0.00608165 0.00502329 0.00987921 0.04187052
 0.06708002 0.14053139 0.18866597 0.24272117 0.33502363]


In [129]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=f_arr, y=C_arr,
    name='Calculated C',
    mode='markers',
    error_y=dict(
        type='data',
        array=C_std_arr,
        color='blue'
    ),
    marker=dict(
        color='red',
        size=4
    )
))

fig.update_layout(
    xaxis_title='Frequency (Hz)',
    yaxis_title='Calibration Constant',
    xaxis=dict(
        type='linear'
    )
)

fig.show()